In [29]:
import sys
from scipy.io import loadmat
import os
import numpy as np

path_str = "/Users/fangfang/Documents/MATLAB/projects/ColorEllipsoids/FilesFromPsychtoolbox"
os.chdir(path_str)

In [79]:
def get_gridPts(X, Y, val_fixed_dim, fixed_dim = list(range(3))):
"""
GET_GRIDPTS Generates grid points for RGB values with one dimension fixed.

This function returns a cell array of grid points for RGB combinations
when one of the R, G, or B dimensions is fixed to a specific value. The
grid points are generated based on the input ranges for the two varying
dimensions.

Parameters:
X - A vector of values for the first varying dimension.
Y - A vector of values for the second varying dimension.
fixed_dim - An array of indices (1 for R, 2 for G, 3 for B) indicating 
            which dimension(s) to fix. Can handle multiple dimensions.
val_fixed_dim - An array of values corresponding to each fixed dimension
                specified in `fixed_dim`. Each value in `val_fixed_dim`
                is used to fix the value of the corresponding dimension.

Returns:
grid_pts - A cell array where each cell contains a 3D matrix of grid 
           points. Each matrix corresponds to a set of RGB values where 
           one dimension is fixed. The size of each matrix is determined 
           by the lengths of X and Y, with the third dimension representing
           the RGB channels.
"""
    #Initialize a cell array to hold the grid points for each fixed dimension.
    grid_pts = [] 
    #Loop through each fixed dimension specified.
    for i in range(len(fixed_dim)):
        #Determine the dimensions that will vary.
        varying_dim = list(set(range(3)) - set([fixed_dim[i]]))
        #Initialize a cell array to hold the current set of grid points.
        grid_pts_i = np.zeros((3, X.shape[0], X.shape[1]))
        #Set the fixed dimension to its specified value across all grid points.
        grid_pts_i[fixed_dim[i],:,:] = val_fixed_dim[i] * np.ones_like(X)
        #Assign the input ranges to the varying dimensions.
        grid_pts_i[varying_dim[0],:,:] = X
        grid_pts_i[varying_dim[1],:,:] = Y
        #Concatenate the individual dimension arrays into a 3D matrix and
        #store it in the output cell array.
        grid_pts.append(grid_pts_i)
        
    return grid_pts

def 

In [38]:
#load data
param = {}
T_cones_mat = loadmat('T_cones.mat')
param['T_cones'] = T_cones_mat['T_cones'] #size: (3, 61)

B_monitor_mat = loadmat('B_monitor.mat')
param['B_monitor'] = B_monitor_mat['B_monitor'] #size: (61, 3)

M_LMSToXYZ_mat = loadmat('M_LMSToXYZ.mat')
param['M_LMSToXYZ'] = M_LMSToXYZ_mat['M_LMSToXYZ'] #size: (3, 3)

In [80]:
#First create a cube and select the RG, the RB and the GB planes
param['nGridPts'] = 100
param['grid'] = np.linspace(0,1,param['nGridPts'])
param['x_grid'], param['y_grid'] = np.meshgrid(param['grid'], param['grid'])

#number of selected planes
param['nPlanes'] = 3

#for RG / RB / GB plane, we fix the B / G / R value to be one of the following
stim = {}
stim['fixed_RGBvec'] = 0.5

#get the grid points for those three planes with one dimension having a specific fixed value
stim['plane_points'] = get_gridPts(param['x_grid'],\
                                   param['y_grid'],\
                                   np.full(3, stim['fixed_RGBvec']))
        

In [83]:
#set a grid for the reference stimulus
#pick 5 x 5 reference points 
stim['nGridPts_ref'] = 5
stim['grid_ref'] = np.linspace(0.2,1,stim['nGridPts_ref'])

stim['x_grid_ref'], stim['y_grid_ref'] = np.meshgrid(stim['grid_ref'],stim['grid_ref'])

#get the grid points for the reference stimuli of each plane
stim['ref_points'] = get_gridPts(stim['x_grid_ref'],\
                                 stim['y_grid_ref'],\
                                 np.full(3, stim['fixed_RGBvec']))

In [ ]:
#visualize the color planes